In [1]:
import json

# Load the Yosys-generated JSON netlist
with open('synthesized_4x4_v4.json', 'r') as f:
    netlist = json.load(f)

# Specify the top module manually (adjust the name as needed)
top_module_name = 'Multiplier_4x4'
if top_module_name not in netlist.get('modules', {}):
    raise KeyError(f"Module {top_module_name} not found in the netlist.")

module = netlist['modules'][top_module_name]

# Build a mapping for port nodes where numbering restarts for each port.
# For each port, we assign new names as f"{port_name}{index}" where index restarts at 0.
port_node_mapping = {}
for port_name, port in module.get("ports", {}).items():
    bits = port.get("bits", [])
    for idx, bit in enumerate(bits):
        new_node_name = f"{port_name}{idx}"  # numbering restarts for each port
        port_node_mapping[str(bit)] = new_node_name

print("Port node mapping:", port_node_mapping)

# Prepare a circuit dictionary with a set for nodes and a list for gates.
circuit = {"nodes": set(), "gates": []}

# Define a mapping from Yosys cell names to the gate types you want.
cell_mapping = {
    "$_AND_": "AND",
    "$_OR_": "OR",
    "$_NOT_": "NOT",
    "$_ANDNOT_": "ANDNOT",
    "$_ORNOT_": "ORNOT",
    "$_NAND_": "NAND",
    "$_NOR_": "NOR",
    "$_XOR_": "XOR",
    "$_XNOR_": "XNOR",
    # Add additional mappings if desired.
}

# Iterate over each cell in the top module.
for cell_name, cell in module.get("cells", {}).items():
    yosys_type = cell["type"]
    gate_type = cell_mapping.get(yosys_type, yosys_type)
    
    gate_nodes = []
    for port, nets in cell.get("connections", {}).items():
        # In Yosys JSON, nets is usually a list (even for a single connection)
        if isinstance(nets, list):
            net = nets[0]
        else:
            net = nets
        net_str = str(net)
        # Use the new name from our port mapping if it exists, otherwise use the raw net.
        node_name = port_node_mapping.get(net_str, net_str)
        gate_nodes.append(node_name)
        circuit["nodes"].add(node_name)
    
    circuit["gates"].append({"type": gate_type, "nodes": gate_nodes})

# Convert the nodes set to a sorted list for reproducibility.
circuit["nodes"] = sorted(list(circuit["nodes"]))

# Write the resulting circuit dictionary to a JSON file.
with open('circuit.json', 'w') as outfile:
    json.dump(circuit, outfile, indent=4)

print("Circuit has been written to circuit.json")


Port node mapping: {'2': 'a0', '3': 'a1', '4': 'a2', '5': 'a3', '6': 'b0', '7': 'b1', '8': 'b2', '9': 'b3', '10': 's0', '11': 's1', '12': 's2', '13': 's3', '14': 's4', '15': 's5', '16': 's6', '17': 's7'}
Circuit has been written to circuit.json


In [8]:
import Scripts.CustomScripts as CS
import Scripts.generate_weight_from_logic as GWL
import json

with open("circuit.json", 'r') as infile:
    circuit = json.load(infile)
print(circuit)
# Preprocess the circuit to ensure that XOR and XNOR gates have the expected number of nodes.
for gate in circuit["gates"]:
    gate_type = gate["type"]
    # Set expected number of nodes based on gate type.
    if gate_type == "XOR":
        expected_nodes = 4  # 3 provided + 1 hidden
    elif gate_type == "XNOR":
        expected_nodes = 5  # 3 provided + 2 hidden
    else:
        continue  # Skip other gate types

    gate_index = circuit["gates"].index(gate)

    # Ensure XOR has a single hidden node appended
    if gate_type == "XOR" and len(gate["nodes"]) == 3:
        hidden_node = f"hidden_XOR_{gate_index}_3"
        gate["nodes"].append(hidden_node)
        if hidden_node not in circuit["nodes"]:
            circuit["nodes"].append(hidden_node)

    # Ensure XNOR has exactly 5 nodes and in correct order
    elif gate_type == "XNOR":
        if len(gate["nodes"]) == 3:  # Initial state: ['A', 'B', 'C']
            # Generate hidden nodes
            hidden_0 = f"hidden_XNOR_{gate_index}_H0"
            hidden_1 = f"hidden_XNOR_{gate_index}_H1"
            
            # Insert H0 at position 2 (before C)
            gate["nodes"].insert(2, hidden_0)
            # Append H1 at the end
            gate["nodes"].append(hidden_1)

            # Ensure the hidden nodes are registered in the circuit's node list
            for hidden_node in [hidden_0, hidden_1]:
                if hidden_node not in circuit["nodes"]:
                    circuit["nodes"].append(hidden_node)

print(circuit)
h_total, J_total, nodes = GWL.generate_circuit_weights(circuit,SaveCircuit=True,CircuitName="8bIntFac_v4")

{'nodes': ['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', 'a0', 'a1', 'a2', 'a3', 'b0', 'b1', 'b2', 'b3', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7'], 'gates': [{'type': 'OR', 'nodes': ['18', '19', '20']}, {'type': 'AND', 'nodes': ['21', '20', 's3']}, {'type': 'NAND', 'nodes': ['22', '21', '23']}, {'type': 'NAND', 'nodes': ['24', '25', '26']}, {'type': 'NAND', 'nodes': ['27', '28', '29']}, {'type': 'NOT', 'nodes': ['29', '30']}, {'type': 'AND', 'nodes': ['a1', 'b3', '31']}, {'type': 'AND', 'nodes': ['b1', 'a3', '32']}, {'type': 'AND', 'nodes': ['b2', 'a3', '33']}, {'type': 'NAND', 'nodes': ['

In [7]:
for gate in circuit["gates"]:
    if gate["type"] == "XOR":
        print(f"XOR Gate Nodes: {gate['nodes']}")
    elif gate["type"] == "XNOR":
        print(f"XNOR Gate Nodes: {gate['nodes']}")

XNOR Gate Nodes: ['36', '32', 'hidden_XNOR_10_H0', '37', 'hidden_XNOR_10_H1']
XNOR Gate Nodes: ['30', '40', 'hidden_XNOR_15_H0', '42', 'hidden_XNOR_15_H1']
XNOR Gate Nodes: ['33', '50', 'hidden_XNOR_27_H0', '54', 'hidden_XNOR_27_H1']
XNOR Gate Nodes: ['49', '54', 'hidden_XNOR_29_H0', '56', 'hidden_XNOR_29_H1']
XNOR Gate Nodes: ['48', '59', 'hidden_XNOR_33_H0', 's5', 'hidden_XNOR_33_H1']
XNOR Gate Nodes: ['69', '70', 'hidden_XNOR_45_H0', 's2', 'hidden_XNOR_45_H1']
XNOR Gate Nodes: ['71', '72', 'hidden_XNOR_46_H0', 's1', 'hidden_XNOR_46_H1']
XNOR Gate Nodes: ['73', '75', 'hidden_XNOR_57_H0', '79', 'hidden_XNOR_57_H1']
XNOR Gate Nodes: ['80', '81', 'hidden_XNOR_62_H0', '70', 'hidden_XNOR_62_H1']
XNOR Gate Nodes: ['76', '34', 'hidden_XNOR_72_H0', '88', 'hidden_XNOR_72_H1']
XNOR Gate Nodes: ['87', '88', 'hidden_XNOR_74_H0', '89', 'hidden_XNOR_74_H1']
XNOR Gate Nodes: ['78', '89', 'hidden_XNOR_76_H0', '90', 'hidden_XNOR_76_H1']
XNOR Gate Nodes: ['85', '90', 'hidden_XNOR_78_H0', '91', 'hidden

In [4]:
import json

# Create a dictionary mapping each node name to its index
node_to_index = {node: idx for idx, node in enumerate(nodes)}

# Load the circuit JSON from a file
with open("circuit.json", "r") as infile:
    circuit = json.load(infile)

# For each gate, map its node names to indices
gate_index_map = []
for gate in circuit["gates"]:
    gate_type = gate["type"]
    mapped_nodes = []
    for node in gate["nodes"]:
        # Look up the node index using the mapping dictionary
        index = node_to_index.get(node)
        if index is None:
            print(f"Warning: Node '{node}' not found in node_to_index mapping.")
        mapped_nodes.append(index)
    gate_index_map.append({
        "gate_type": gate_type,
        "node_indices": mapped_nodes
    })

# Print the mapping in a formatted JSON style
print(json.dumps(gate_index_map, indent=4))


[
    {
        "gate_type": "AND",
        "node_indices": [
            8,
            0,
            16
        ]
    },
    {
        "gate_type": "AND",
        "node_indices": [
            1,
            8,
            24
        ]
    },
    {
        "gate_type": "AND",
        "node_indices": [
            9,
            0,
            25
        ]
    },
    {
        "gate_type": "XOR",
        "node_indices": [
            25,
            24,
            17
        ]
    },
    {
        "gate_type": "NAND",
        "node_indices": [
            2,
            8,
            26
        ]
    },
    {
        "gate_type": "NAND",
        "node_indices": [
            9,
            1,
            27
        ]
    },
    {
        "gate_type": "XNOR",
        "node_indices": [
            27,
            26,
            28
        ]
    },
    {
        "gate_type": "NAND",
        "node_indices": [
            10,
            0,
            29
        ]
    },
    {
       